In [1]:
import tensorflow as tf
import numpy as np
import os, time, random
from utils import str2bool, get_logger, get_entity, get_entitys,get_x_entity
from data import random_embedding,tag2label
import datapreprocessing as dp
from model import BiLSTM_CRF

C:\Users\Ian\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
## Session configuration
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # default: 0
config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.2  # need ~700MB GPU memory

In [3]:
## paths setting
paths = {}
timestamp='1529463214'
output_path = os.path.join('.', 'data_path'+"_save", timestamp)
if not os.path.exists(output_path): os.makedirs(output_path)
summary_path = os.path.join(output_path, "summaries")
paths['summary_path'] = summary_path
if not os.path.exists(summary_path): os.makedirs(summary_path)
model_path = os.path.join(output_path, "checkpoints/")
if not os.path.exists(model_path): os.makedirs(model_path)
ckpt_prefix = os.path.join(model_path, "model")
paths['model_path'] = ckpt_prefix
result_path = os.path.join(output_path, "results")
paths['result_path'] = result_path
if not os.path.exists(result_path): os.makedirs(result_path)
log_path = os.path.join(result_path, "log.txt")
paths['log_path'] = log_path

In [4]:
## get char embeddings
word2id = dp.loadVocab('word2id.pkl')

vocab_size: 1234


In [ ]:
embeddings = np.load('pretrain_embedding.npy')

In [ ]:
## demo
ckpt_file = tf.train.latest_checkpoint(model_path)
print(ckpt_file)
paths['model_path'] = ckpt_file
model = BiLSTM_CRF(embeddings, tag2label, word2id, paths, config=config)
model.build_graph()
saver = tf.train.Saver()
with tf.Session(config=config) as sess:
    print('============= demo =============')
    saver.restore(sess, ckpt_file)
    while(1):
        print('Please input your sentence:')
        demo_sent = input()
        if demo_sent == '' or demo_sent.isspace():
            print('See you next time!')
            break
        else:
            demo_sent = list(demo_sent.strip())
            demo_data = [(demo_sent, ['O'] * len(demo_sent))]
            tag = model.demo_one(sess, demo_data)
            print(tag)
            ZS = get_entity(tag, demo_sent)
            print('ZS: {}'.format(ZS))
            print(get_x_entity('zs',tag, demo_sent))
            d = get_entitys(list(dp.tags),tag, demo_sent)
            print(d)
#             PER, LOC, ORG = get_entity(tag, demo_sent)
#             print('PER: {}\nLOC: {}\nORG: {}'.format(PER, LOC, ORG))

.\data_path_save\1529463214\checkpoints/model-1200


C:\Users\Ian\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


============= demo =============
INFO:tensorflow:Restoring parameters from .\data_path_save\1529463214\checkpoints/model-1200


Restoring parameters from .\data_path_save\1529463214\checkpoints/model-1200


Please input your sentence:
患者三周无明显诱因下出现心悸、胸闷，无胸痛、气急等不适，面色黧黑，胃纳可，二便调，夜寐安。
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'B-ZS', 'I-ZS', 0, 'B-ZS', 'I-ZS', 0, 0, 'B-1ZS', 'I-1ZS', 0, 'B-1ZS', 'I-1ZS', 0, 0, 0, 0, 'B-YC', 'I-YC', 'I-YC', 'I-YC', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
ZS: ['心悸', '胸闷']
['心悸', '胸闷']
{'1b': [], '1jc': [], '1jx': [], '1m': [], '1o': [], '1qt': [], '1s': [], '1t': [], '1w': [], '1x': [], '1yc': [], '1yw': [], '1zl': [], '1zs': ['胸痛', '气急'], '2b': [], '2jc': [], '2jx': [], '2m': [], '2o': [], '2qt': [], '2s': [], '2t': [], '2w': [], '2x': [], '2yc': [], '2yw': [], '2zl': [], '2zs': [], '3b': [], '3jc': [], '3jx': [], '3m': [], '3o': [], '3qt': [], '3s': [], '3t': [], '3w': [], '3x': [], '3yc': [], '3yw': [], '3zl': [], '3zs': [], '4b': [], '4jc': [], '4jx': [], '4m': [], '4o': [], '4qt': [], '4s': [], '4t': [], '4w': [], '4x': [], '4yc': [], '4yw': [], '4zl': [], '4zs': [], '5b': [], '5jc': [], '5jx': [], '5m': [], '5o': [], '5qt': [], '5s': [], '5t': [], '5w': [], '5x'

In [7]:
with tf.Session(config=config) as sess:
    print('============= demo =============')
    saver.restore(sess, ckpt_file)
    corpus_dir = 'D:\\Desktop\\产品\\辅诊\\cndata\\ru1_tb_cis_mzdzbl\\tt'
    for s in os.listdir(corpus_dir):
        if s.endswith('.csv'):
            with open(os.path.join(corpus_dir,s),'r',encoding='utf8') as f:
                l = f.readlines()
            with open(os.path.join(corpus_dir,'marked{}'.format(s)),'a',encoding='utf8') as f:
                for demo_sent in l:
                    if demo_sent:
                        demo_sent = list(demo_sent.strip())
                        demo_data = [(demo_sent, ['O'] * len(demo_sent))]
                        tag = model.demo_one(sess, demo_data)
                        count = 0
                        mark = 0
                        for i in range(len(tag)):
                            if tag[i] == 0:
                                tag[i] = 'O'
                            if mark == 0:
                                if tag[i].startswith('B'):
                                    if i != 0 and tag[i-1]=='O':
                                        demo_sent.insert(i+count,'/o')
                                        count += 1
                                        mark = 1
                            elif mark == 1:
                                if tag[i].startswith('B'):
                                    demo_sent.insert(i+count,'/{}'.format(tag[i-1][2:].lower()))
                                    count += 1
                                elif tag[i].startswith('O'):
                                    demo_sent.insert(i+count,'/{}'.format(tag[i-1][2:].lower()))
                                    count += 1
                                    mark = 0
                        f.write(''.join(demo_sent)+'\n')

============= demo =============
INFO:tensorflow:Restoring parameters from .\data_path_save\1529463214\checkpoints/model-1200


Restoring parameters from .\data_path_save\1529463214\checkpoints/model-1200


In [6]:
test_path = os.path.join('.', 'data_path', 'test_data')
test_data = read_corpus(test_path)
test_size = len(test_data)

## testing model
# elif args.mode == 'test':
#     ckpt_file = tf.train.latest_checkpoint(model_path)
#     print(ckpt_file)
#     paths['model_path'] = ckpt_file
#     model = BiLSTM_CRF(args, embeddings, tag2label, word2id, paths, config=config)
#     model.build_graph()
#     print("test data: {}".format(test_size))
#     model.test(test_data)
## testing model
ckpt_file = tf.train.latest_checkpoint(model_path)
print(ckpt_file)
paths['model_path'] = ckpt_file
model = BiLSTM_CRF(embeddings, tag2label, word2id, paths, config=config)
model.build_graph()
print("test data: {}".format(test_size))
model.test(test_data)

model_path